# About this notebook 

#### Feature: Description

This notebook employs Word2Vec with CountVectorizer to create a training and test dataset of the pet description field.  Then the datasets are used in the XGBoost algorithm to create an adoption model.

<div class="span5 alert alert-success">
<p> <I> Feature Description: </I> The "Description" data is a profile write-up for each pet.
     <br>
    <I> Source: </I> https://www.kaggle.com/c/petfinder-adoption-prediction/data  </p>
</div>

<div class="span5 alert alert-success">
<p> <I> Predictor (Adoption Speed) Description: </I> 

Contestants are required to predict this value. The value is determined by how quickly, if at all, a pet is adopted.   
<br> 
The values are determined in the following way:   
0 - Pet was adopted on the same day as it was listed.    
1 - Pet was adopted between 1 and 7 days (1st week) after being listed.    
2 - Pet was adopted between 8 and 30 days (1st month) after being listed.    
3 - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.    
4 - No adoption after 100 days of being listed.    

</p>
</div>

<div class="span5 alert alert-info">
<p> <B>  Imports and Data Loading: </B>  </p>
</div>

In [45]:
#Imports
import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.ensemble import VotingClassifier

In [46]:
import warnings
warnings.filterwarnings('ignore')

%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject2Repository\09 PetfindersData\TrainingData

C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject2Repository\09 PetfindersData\TrainingData


In [47]:
#Import the csv file

dfi = pd.read_csv('milestonereport2_petdata_withnooutliers.csv',index_col=0)

#remove the rows with no description 
dfi = dfi[pd.notnull(dfi.Description)]

<div class="span5 alert alert-info">
<p> <B>  NLP using Word2Vec and XGBoost: </B>  </p>
</div>

In [48]:
#Create dataframe of pet description feature
dfd = dfi[['Description','AdoptionSpeed']]
dfd.columns = ['description', 'adoptionspeed']

In [49]:
#Tokenize and lemmatize the description data
mylist = []
for index, row in dfd.iterrows():
    
    #mylist = row[0]
 
    #split sentence into words
    tokens = nltk.word_tokenize(str(row[0]))
    
    #remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    
    #convert the tokens to lowercase
    wordslc = [word.lower() for word in words]
    
    mylist.append(wordslc)


#print(wordslc)
dfd['tokenized_desc'] = mylist
dfd.head(1)

,description,adoptionspeed,tokenized_desc
0,Milo went missing after a week with her new ad...,3,"[milo, went, missing, after, a, week, with, he..."


In [50]:
#Convert tokenized desc from list to string
dfd['tokenized_desc_string'] = str(mylist).strip('[]')
dfd.head(1)

,description,adoptionspeed,tokenized_desc,tokenized_desc_string
0,Milo went missing after a week with her new ad...,3,"[milo, went, missing, after, a, week, with, he...","'milo', 'went', 'missing', 'after', 'a', 'week..."


In [51]:
count_vect = CountVectorizer()
count_vect.fit(dfd['description'])

train_description = count_vect.transform(dfd['description'].values)

X = train_description
Y = dfd['adoptionspeed'].values

In [52]:
#Create a training and test data set
test_size = 0.33
seed = 7
X_train_nlp, X_test_nlp, Y_train_nlp, Y_test_nlp = train_test_split(X, Y, test_size=test_size,
random_state=seed)

In [53]:
#XGBoost: Fit the model using default values
model = XGBClassifier()

model.fit(X_train_nlp,Y_train_nlp)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [54]:
#XGBoost: Predict the labels of the test set based on default values
Y_pred_nlp = model.predict(X_test_nlp)

Y_pred_proba_nlp = model.predict_proba(X_test_nlp)

In [55]:
print('CLASSIFICATION REPORT FOR DEFAULT VALUES')
print(classification_report(Y_test_nlp, Y_pred_nlp))

print()

print('ACCURACY SCORE FOR DEFAULT VALUES')
print(accuracy_score(Y_test_nlp,Y_pred_nlp))

CLASSIFICATION REPORT FOR DEFAULT VALUES
             precision    recall  f1-score   support

          0       0.67      0.01      0.03       149
          1       0.38      0.08      0.14      1022
          2       0.32      0.38      0.35      1291
          3       0.40      0.11      0.17      1092
          4       0.35      0.72      0.47      1385

avg / total       0.37      0.35      0.29      4939


ACCURACY SCORE FOR DEFAULT VALUES
0.34561652156306943


<div class="span5 alert alert-info">
<p> <B>  Run the 20 Features (non-NLP) using XGBoost with best parameters identified in Milestone Report Part 2b: </B>  </p>
</div>

In [56]:
#Drop the columns that are not needed for machine learning
dfm = dfi.drop(['Name','RescuerID','Description','PetID'],axis=1)
dfm.head(1)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
0,1,2,0,26,2,2,0,0,2,1,1,1,2,1,1,0,41326,0,3,3


In [57]:
#Create the array
array = dfm.values
X = array[:,0:19]
Y = array[:,19]

In [58]:
#Create a training and test data set
test_size = 0.33
seed = 7
X_train_nonnlp, X_test_nonnlp, Y_train_nonnlp, Y_test_nonnlp = train_test_split(X, Y, test_size=test_size,
random_state=seed)

In [59]:
#XGBoost: Fit the model using best hyper parameter tuning results from Part 2b
model = XGBClassifier(colsample_bylevel = 0.5, colsample_bytree = 0.5, learning_rate = 0.1, max_depth = 6,
                      min_child_rate = 0, subsample = 1)
                      
model.fit(X_train_nonnlp,Y_train_nonnlp)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_rate=0, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [60]:
#XGBoost: Predict the labels of the test set
Y_pred_nonnlp = model.predict(X_test_nonnlp)

Y_pred_proba_nonnlp = model.predict_proba(X_test_nonnlp)

In [61]:
print('CLASSIFICATION REPORT')
print(classification_report(Y_test_nonnlp, Y_pred_nonnlp))

print()

print('ACCURACY SCORE')
print(accuracy_score(Y_test_nonnlp,Y_pred_nonnlp))

CLASSIFICATION REPORT
             precision    recall  f1-score   support

          0       0.50      0.01      0.03       149
          1       0.38      0.33      0.35      1022
          2       0.34      0.44      0.38      1291
          3       0.39      0.18      0.24      1092
          4       0.48      0.65      0.55      1385

avg / total       0.40      0.40      0.38      4939


ACCURACY SCORE
0.4041303907673618


<div class="span5 alert alert-info">
<p> <B>  Stacking Model: Create an ensemble model that combines the NLP model with the non-NLP (20 Features) model </B>  </p>
</div>

In [62]:
#Create a data frame of the first models nlp and nonnlp predicted results.  Then transpose it.
df_smf = pd.DataFrame([Y_pred_nlp, Y_pred_nonnlp, Y_test_nlp]).transpose()

df_smf.columns = ['firstmodel_nlp_predicted','firstmodel_nonnlp_predicted', 'firstmodel_Y_values']

df_smf.head(1)


,firstmodel_nlp_predicted,firstmodel_nonnlp_predicted,firstmodel_Y_values
0,4,4,4


In [63]:
#Create the array for the stacked model
array = df_smf.values
X = array[:,0:1]
Y = array[:,1]

In [64]:
#Create a train and test dataset to use on the stacked model
test_size = 0.33
seed = 7
X_train_stacked, X_test_stacked, Y_train_stacked, Y_test_stacked = train_test_split(X, Y, test_size=test_size,
random_state=seed)

In [65]:
#XGBoost: Fit the model
model = XGBClassifier()
                      
model.fit(X_train_stacked,Y_train_stacked)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [66]:
#XGBoost: Predict the labels of the test set
Y_pred_stacked = model.predict(X_test_stacked)

Y_pred_proba_stacked = model.predict_proba(X_test_stacked)

In [67]:
print('CLASSIFICATION REPORT')
print(classification_report(Y_test_stacked, Y_pred_stacked))

print()

print('ACCURACY SCORE')
print(accuracy_score(Y_test_stacked,Y_pred_stacked))

CLASSIFICATION REPORT
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.00      0.00      0.00       285
          2       0.41      0.46      0.44       544
          3       0.37      0.21      0.27       183
          4       0.46      0.69      0.56       616

avg / total       0.36      0.44      0.39      1630


ACCURACY SCORE
0.4392638036809816
